In [2]:
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import random as rd

In [47]:
class SimpleMultiClassBoosting(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator=None, n_estimators=50):
        self.base_estimator = base_estimator if base_estimator is not None else DecisionTreeClassifier(max_depth=1)
        self.n_estimators = n_estimators
        self.learners = np.empty(n_estimators, dtype=object)
        self.learner_weights = np.empty(n_estimators, dtype=float)
        self.samples_weights = None
        self.label_encoder = LabelEncoder()

    # def fit(self, X, y):
    #     Y = self.label_encoder.fit_transform(y=y)
    #     n_classes = len(np.unique(Y))
    #     indexes = np.arange(X.shape[0])
    #     self.samples_weights = (1/X.shape[0]) * np.ones(X.shape[0] ,dtype=float)
    #     for i in range(self.n_estimators):
    #         learner = self.base_estimator
    #         learner.fit(X, Y)
    #         self.learners[i] = learner
    #         missClassifiedIndexes = []
    #         predictions = learner.predict(X=X)
    #         for j in range(X.shape[0]):
    #             if predictions[j] != Y[j]:
    #                 missClassifiedIndexes.append(j)
    #         learner_error = (np.sum(self.samples_weights[missClassifiedIndexes])) / np.sum(self.samples_weights)
    #         learner_rate = 0
    #         if learner_error == 0:
    #             learner_rate = np.inf
    #         else:            
    #             learner_rate = np.log((1-learner_error) / learner_error) + np.log(n_classes - 1)
    #         if learner_error >= 1 - (1 / n_classes):
    #             learner_rate = 0
    #         self.learner_weights[i] = learner_rate 
    #         for idx in missClassifiedIndexes:
    #             self.samples_weights[idx] *= self.samples_weights[idx]*np.exp(learner_rate)
    #         self.samples_weights /= np.sum(self.samples_weights)
    def fit(self, X, y):
        Y = self.label_encoder.fit_transform(y=y)
        n_classes = len(np.unique(Y))
        indexes = np.arange(X.shape[0])
        self.samples_weights = (1/X.shape[0]) * np.ones(X.shape[0], dtype=float)
        for i in range(self.n_estimators):
            learner = self.base_estimator
            # training_indexes = np.array(rd.choices(indexes, weights=self.samples_weights, k=X.shape[0]))
            # sx = X[training_indexes]
            # sy = Y[training_indexes]
            # uy = np.unique(sy)
            
            # # if uy.shape[0] < n_classes:
            #     # unseen = list(set(np.unique(Y)) - set(uy))
            #     # for u in unseen:
            #         # ui = rd.choices(np.where(Y == u)[0], k=1)
            #         # sx = np.vstack([sx, X[ui]])
            #         # sy = np.hstack([sy, Y[ui]])
            # # 
            learner.fit(X, Y)
            self.learners[i] = learner
            
            predictions = learner.predict(X)
            misclassified = predictions != Y
            learner_error = np.sum(self.samples_weights[misclassified]) / np.sum(self.samples_weights)
            
            learner_rate = np.log((1 - learner_error) / learner_error) + np.log(n_classes - 1)
            if learner_error >= 1 - 1/n_classes:
                learner_rate = 0
            # if learner_error == 0:
                # learner_rate = np.inf
            # else:
                # learner_rate = np.log((1 - learner_error) / learner_error) + np.log(n_classes - 1)
            
            self.learner_weights[i] = learner_rate
            
            if learner_error < 1/n_classes:
                for idx in range(X.shape[0]):
                    self.samples_weights[idx] *= np.exp(learner_rate) if misclassified[idx] else 1.0
            # for j in misclassified:
            #     self.samples_weights[j] *= np.exp(learner_rate)
            
            self.samples_weights /= np.sum(self.samples_weights)

    def predict(self, X):
        predictionsOfLearners = []
        for learner in self.learners:
            if learner != None:
                predictionsOfLearners.append(learner.predict(X))
        predictionsOfLearners = np.array(predictionsOfLearners)
        prediction = np.empty(X.shape[0])        
        for i in range(X.shape[0]):
            labels = np.unique(predictionsOfLearners[:, i])
            votes = {label : 0 for label in labels}
            for j in range(len(predictionsOfLearners[:, i])):
                for label in labels:
                    if predictionsOfLearners[j, i] == label:
                        votes[label] += self.learner_weights[j]
            finalPrediction = max(votes, key=votes.get)
            prediction[i] = self.label_encoder.inverse_transform(np.array([finalPrediction]))
        return prediction

In [32]:
iris = load_iris()
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], columns=iris['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:, :-1].to_numpy(), df.iloc[:, -1].to_numpy(), test_size=0.2, random_state=42)

In [48]:
m = SimpleMultiClassBoosting()
m.fit(X_train, Y_train)

In [49]:
prediction = m.predict(X_test)
prediction

array([1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.])

In [50]:
print(m.learner_weights)

[1.42403469e+00 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15 2.66453526e-15 2.66453526e-15
 2.66453526e-15 2.66453526e-15]


In [51]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, prediction))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       0.45      1.00      0.62         9
         2.0       0.00      0.00      0.00        11

    accuracy                           0.63        30
   macro avg       0.48      0.67      0.54        30
weighted avg       0.47      0.63      0.52        30



C:\Users\javadpsk\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\javadpsk\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\javadpsk\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [114]:
a = DecisionTreeClassifier(max_depth=3)
a.fit(X_train, Y_train)
pred = a.predict(X_test)
print(classification_report(Y_test, pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      1.00      1.00         9
         2.0       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

